<a href="https://colab.research.google.com/github/davipguimaraes/Imers-o-de-dados---alura/blob/main/02_03_Imers%C3%A3o_Python_Tabela_de_a%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imerção Aula 02

In [33]:
import pandas as pd
import numpy as np
import plotly.express as px

In [34]:
h_slug_acao = "Ativo"
h_ultimo_valor = "Último (R$)"
h_variacao_dia = "Var. Dia (%)"
h_variacao_sem = "Var. Sem. (%)"
h_variacao_mes = "Var. Mês (%)"
h_variacao_ano = "Var. Ano (%)"
h_variacao_12m = "Var. 12M (%)"
h_variacao_min = "Val. Mín"
h_variacao_max = "Val. Máx"

h_contagem_acoes = "Qtde. Teórica"
h_nome = "Nome"
h_segmento = "Segmento"
h_idade = "Idade (anos)"

In [35]:
df_acoes = pd.read_excel("/content/acoes-b3.xlsx", sheet_name="Semente")
df_total_acoes = pd.read_excel("/content/acoes-b3.xlsx", sheet_name="Total_de_acoes")
df_ticker = pd.read_excel("/content/acoes-b3.xlsx", sheet_name="Ticker")
df_institucional = pd.read_excel("/content/acoes-b3.xlsx", sheet_name="ChatGPT")

In [36]:
# Combinando dados
df_acoes_e_total = pd.merge(df_acoes,df_total_acoes,how="left", left_on=[h_slug_acao],right_on=['Código'])

# df_acoes_total_ticker = pd.merge(df_acoes_e_total,df_ticker,how="left", left_on=[h_slug_acao],right_on=['Ticker'])
df_institucional_ticker_temp = pd.merge(df_ticker,df_institucional,how="left", left_on=['Nome'],right_on=['Nome da empresa'])
df_institucional_ticker = df_institucional_ticker_temp.drop_duplicates()

# DataFramePrincipal
df_main_acoes = pd.merge(df_acoes_e_total,df_institucional_ticker,how="left", left_on=[h_slug_acao],right_on=['Ticker'])

# Imerção Aula 03

In [37]:
def get_group_idade(idade):
  if( idade < 25):
    return '0-25'
  if( idade < 50):
    return '25-50'
  if( idade < 75):
    return '50-75'
  if( idade < 100):
    return '75-100'
  return "100..."

def get_result_per_value_variation(valor):
  if( valor > 0):
    return 'subiu'
  if( valor < 0 ):
    return "desceu"
  return "estável"

df_main_acoes['grupo_idade'] = df_main_acoes[h_idade].apply(get_group_idade)

def calcule_metricas_acoes(df_dataset,variacao,prefixo):
  df_dataset[f'{prefixo}_variacao_percentual'] = df_dataset[variacao] / 100
  df_dataset[f'{prefixo}_valor_inicial'] = df_dataset[h_ultimo_valor]  / (1+df_dataset[f'{prefixo}_variacao_percentual'])
  df_dataset[f'{prefixo}_valor_variacao'] = df_dataset[f'{prefixo}_variacao_percentual']*df_dataset[f'{prefixo}_valor_inicial']*df_dataset[h_contagem_acoes]
  df_dataset[f'{prefixo}_result'] = df_dataset[f'{prefixo}_valor_variacao'].apply(get_result_per_value_variation)
  return


# calculando metricas dia/semana/mes/ano/12meses/min/max
calcule_metricas_acoes(df_main_acoes,h_variacao_dia,'dia')
calcule_metricas_acoes(df_main_acoes,h_variacao_sem,'sem')
calcule_metricas_acoes(df_main_acoes,h_variacao_mes,'mes')
calcule_metricas_acoes(df_main_acoes,h_variacao_ano,'ano')
calcule_metricas_acoes(df_main_acoes,h_variacao_12m,'12m')
calcule_metricas_acoes(df_main_acoes,h_variacao_min,'min')
calcule_metricas_acoes(df_main_acoes,h_variacao_max,'max')

In [38]:
# analises de Max/min/Media/Media dos que subiram/Media dos que desceram

def calcule_metricas_acoes(df_dataset,prefixo):
  analysis = {
    "min": df_dataset[f'{prefixo}_valor_variacao'].max(),
    "max": df_dataset[f'{prefixo}_valor_variacao'].min(),
    "media": df_dataset[f'{prefixo}_valor_variacao'].mean(),
    "media_de_subida": df_dataset.loc[(df_dataset[f'{prefixo}_result'] == 'subiu' ) ,f'{prefixo}_valor_variacao'].mean(),
    "media_de_descida": df_dataset.loc[(df_dataset[f'{prefixo}_result'] == 'desceu' ) ,f'{prefixo}_valor_variacao'].mean(),
  }
  return analysis

calcule_metricas_acoes(df_main_acoes,'dia')
# faça aqui as analizes

{'min': 4762926995.248081,
 'max': -1807432634.4595451,
 'media': 165190210.47934023,
 'media_de_subida': 448164250.23049796,
 'media_de_descida': -181109141.75186732}

In [39]:
# Analises por segmento
df_analysis_segment = df_main_acoes.groupby(h_segmento)['dia_valor_variacao'].sum().reset_index()
df_analysis_segment.head()

,Segmento,dia_valor_variacao
0,Aeronáutica,-2.336519e+08
1,Agronegócio,-9.468664e+06
2,Alimentos,4.078337e+08
3,Aluguel de Carros,-1.807433e+09
4,Automação,-1.182304e+08


In [40]:
# Analises de resultado
df_analysis_resultado = df_main_acoes.groupby('dia_result')['dia_valor_variacao'].sum().reset_index()
df_analysis_resultado.head()

,dia_result,dia_valor_variacao
0,desceu,-6.338820e+09
1,estável,0.000000e+00
2,subiu,1.971923e+10


In [41]:
# Analises de idade
df_analysis_idade = df_main_acoes.groupby('grupo_idade')['dia_valor_variacao'].sum().reset_index()
df_analysis_idade.head()

,grupo_idade,dia_valor_variacao
0,0-25,4.262721e+09
1,100...,-2.024083e+08
2,25-50,-1.883582e+09
3,50-75,4.327439e+09
4,75-100,6.876238e+09


In [42]:
# grafico de resultado
fig = px.bar(df_analysis_resultado, x="dia_result", y="dia_valor_variacao", text="dia_valor_variacao", title="Variação Reais x resutlado")
fig.show()

In [43]:
# grafico de resultado
fig = px.pie(df_analysis_segment, names=h_segmento, values="dia_valor_variacao", title="Variação Reais x segmento")
fig.show()

In [44]:
# grafico de idade
fig = px.bar(df_analysis_idade, x='grupo_idade', y="dia_valor_variacao", title="Variação Reais x idade")
fig.show()